# Day 24
## Part 1
Manual compilation time. Each section of code takes a digit into `w` and is then one of two forms. Either
```
inp w
mul x 0
add x z
mod x 26 # set x to z mod 26
div z 1  # do nothing
add x 10 # set x to z mod 26 + 10
eql x w  # set x to 0, as 1 <= w <= 9
eql x 0  # set x to 1
mul y 0
add y 25
mul y x
add y 1  # set y to 26
mul z y  # multiply z by 26
mul y 0
add y w
add y 2
mul y x
add z y  # add w + 2 to z
```
or
```
inp w
mul x 0
add x z
mod x 26 # set x to z mod 26
div z 26 # set z to z/26 - x is now the remainder
add x -7 # subtract 7 from x
eql x w  # check if x is equal to w
eql x 0  # if it is set x to 0 else 1
mul y 0
add y 25
mul y x  
add y 1  # set y to 25x + 1 i.e. 1 or 26
mul z y  # multiply z by y
mul y 0  
add y w
add y 3  # set y to w + 3
mul y x  # multiply by x
add z y  # add to z
```
The first form can be seen as pushing a number onto a stack using modular arithmetic (modulo 26) and the second pops it. So if we only had these chunks of code, `z` would be `w1` (the first input character) + 2. We want `w1 + 2 - 7 == w2` so `x` is zero and does not add anything to `z` at the end. Therefore the highest value that would apply to is `w1 == 9` and `w2 == 4`.

In general, for push/pop pairs, we want to find the maximum `w1` and `w2` so that `w1 + a == w2 + b`, where a is the value added in the 16th line of the first block, and b is the value subtracted in the 6th line of the second block. 

In [2]:
code = open('input', 'r').read().splitlines()
code[5::18]

['add x 10',
 'add x 15',
 'add x 14',
 'add x 15',
 'add x -8',
 'add x 10',
 'add x -16',
 'add x -4',
 'add x 11',
 'add x -3',
 'add x 12',
 'add x -7',
 'add x -15',
 'add x -7']

In [4]:
pop_values = [int(line.split()[2]) for line in code[5::18]]
push_values = [int(line.split()[2]) for line in code[15::18]]

In [5]:
pop_values

[10, 15, 14, 15, -8, 10, -16, -4, 11, -3, 12, -7, -15, -7]

In [6]:
push_values

[2, 16, 9, 0, 1, 12, 6, 6, 3, 5, 9, 3, 2, 3]

Get the value of interest depending on whether or not it is a pop or push operation.

In [7]:
values = [pop if pop < 0 else push for pop, push in zip(pop_values, push_values)]

In [8]:
values

[2, 16, 9, 0, -8, 12, -16, -4, 3, -3, 9, -7, -15, -7]

In [9]:
def highest_ws(a, b):
    if a > b:
        return (9 + b - a, 9)
    else:
        return (9, 9 + a - b)
    
highest_ws(2, 7)

(9, 4)

There has to be a better way of reconstructing the number than a dictionary where the keys are the index to the position of the digit in the number but I can't think of it right now.

In [19]:
n = {}
stack = []

for i, x in enumerate(values):
    if x >= 0:
        stack.append((x, i))
    else:
        a, j = stack.pop()
        w1, w2 = highest_ws(a, -x)
        n[j] = w1
        n[i] = w2

In [20]:
n

{3: 9,
 4: 1,
 5: 9,
 6: 5,
 2: 4,
 7: 9,
 8: 9,
 9: 9,
 10: 7,
 11: 9,
 1: 8,
 12: 9,
 0: 9,
 13: 4}

In [21]:
''.join(str(n[i]) for i in range(len(n)))

'98491959997994'

## Part 2

In [22]:
def lowest_ws(a, b):
    if a > b:
        return (1, 1 + a - b)
    else:
        return (1 + b - a, 1)
    
lowest_ws(2, 7)

(6, 1)

In [23]:
n = {}
stack = []

for i, x in enumerate(values):
    if x >= 0:
        stack.append((x, i))
    else:
        a, j = stack.pop()
        w1, w2 = lowest_ws(a, -x)
        n[j] = w1
        n[i] = w2

In [24]:
''.join(str(n[i]) for i in range(len(n)))

'61191516111321'

In [ ]:
def run(s):
    answer = []
    reg = {'x': 0, 'y': 0, 'z': 0}
    code_blocks = s.split('inp w\n')
    for block in code_blocks:
        for line in block.strip().splitlines():
            fields = line.split()
            if fields[0] == 'add':
                reg[fields[1]] = 